In [ ]:
## Imports. This contains some imports not required for this notebook. 

import sys
import os
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import pylab                   # Needed to plot histograms.
from astropy.io import fits                    # Need this if you want to use astropy.io io objects.
from astropy.stats import mad_std              # The median absolute deviation, a more robust estimator than std.

'''Filter out warnings. May or may not need this. But shouldn't hurt to put it in.
This is the "new" code for avoiding warnings-- seems to work better. Copied from an SDSS notebook.'''
import warnings
warnings.filterwarnings('ignore')

# Task 1. Creating Faulty Non-Noisy Pixels Fits

In [ ]:
# Specify the path to the data directory.
# datapath = '/Users/alex/_observing/24-inch/2018/manual_dark/darks_180420'
datapath = r'C:\Users\owenm\Documents\ASTR 21200 2019\Data\Project\dark'
savepath = r'C:\Users\owenm\Documents\ASTR 21200 2019\Data\BadPixelsFitss'

In [ ]:
## List files in the the data directory.

whichpath = datapath

## For ALL the files in the directory.
# allfiles = [f for f in os.listdir(datapath)]

## Various list comprehensions can pick out files with particular characteristics.
allfiles = [f for f in os.listdir(whichpath) if '.fit' in f and ('bias.' in f) and 'mdark' not in f]

allfiles = sorted(allfiles)       ## This is necessary on my Mac, may not be for others?
for i in range(len(allfiles)):
    print( i, allfiles[i])

In [ ]:
## Construct a list of files you wish to view and/or process.

acceptlist = True             # True if you want to accept all the files in allfiles.
contiguous = True             # True if you want to accept a contiguous subset of allfiles.
startfile, endfile = 0,9      # The first and last files in a contiquous subset.
flist = [0,3,4,5,6,7,8]        # An explicit list of the files you want to accept.

if acceptlist == True:
    files = allfiles
    for i in range(len(files)):
        print(i, files[i])
else:
    if contiguous == True:
        files = allfiles[startfile:endfile+1]
        for i in range(len(files)):
            print( i, files[i])
    else:
        files = []
        for i in range(len(flist)):
            files.append(allfiles[flist[i]])
        for i in range(len(files)):
            print( i, flist[i], files[i])

In [ ]:
'''Make a stack of images, a list of headers from those images, and calculate some medians and stds
that will help set autoscaling parameters.'''


image = np.zeros((len(files), 1024,1024))
imedian = np.zeros((len(files)))          
istd = np.zeros((len(files)))              
madstd = np.zeros((len(files))) 
headlist = []  
print('image.shape =', image.shape)
for i in range(len(files)):


#############################################################################
    # Use this code block if you want to use the standard astropy.io package.
    fitsfilename = os.path.join(datapath, files[i])    # Full path to a fitsfile.
    hdulist = fits.open(fitsfilename)                  # Open a fits file as an hdulist object.
    hdu0 = hdulist[0]                                  # Define a fits object as the 0th hdu in the fitsfile.
    image[i] = hdu0.data * 1.0                         # Define image in stack as float of data in the 0th hdu.
    headlist.append(hdu0.header)                       # Append the header of the fits object to the header list.
    print('')
    print(i,files[i])
###############################################################################    
    
    ## Calculate some medians and stds to use in autoscaling that aren't unduly biased by extreme values.
    ## Optionally, print out masked and unmasked values to explore the effects of extreme values on the statistics.
    
    madstd[i] = mad_std(image[i],ignore_nan=True)
    imedian[i] = np.nanmedian(image[i])
    istd[i] = np.nanstd(image[i])

In [ ]:
bad = 0
for i in range(len(image)):
    istand = image[i] - imedian[i]
    bad += (istand > istd[i])
    bad += (istand < -istd[i])
for j in range(len(sum(bad))):
    print(str(j) + ':' + str(sum(bad)[j]))

In [ ]:
#Creating a map of how often the pixels have been bad

bads = np.zeros((1024,1024))

imedian2 = np.zeros((len(files)))           # 1D numpy array to hold array medians.
istd2 = np.zeros((len(files)))              # 1D numpy array to hold array stds.

for i in range(len(files)):
    imedian2[i] = np.nanmedian(image[i,200:,:])
    istd2[i] = np.nanstd(image[i,200:,:])

for i in range(len(image)):
    istand = image[i] - imedian2[i]
    bad1 = (istand > istd2[i]) * 1
    bad2 = (istand < -istd2[i]) * 1
    bad = bad1 + bad2
    bads += bad
    
#print(bads)
#print(bads[:,1:])
b = bads[:, 200:]
#print(b)
pix = np.zeros((1024,824))
ctr = 0
for i in range(1024):
    for j in range(824):
        if b[i,j] >= 20:
            ctr += 1
            pix[i,j] = 1
            #print(i,j)
print(str(ctr/(1024 * 824)))

#print(pix)

The above array gives the number of times, for each pixel, it has been deemed "bad" in the list of 300 darks.

If we check pixels that have been bad at least once: 49.2% of pixels meet this criteria If we check pixels that have been bad at least twice: 18.5% of pixels meet this criteria If we check pixels that have been bad at least 3 times: 8.28% of pixels meet this criteria If we check pixels that have been bad at least 4 times: 5.21% of pixels meet this criteria ...

In [ ]:
#Creating a 2D array that is true false values with 1,0 but with corners missing and checking local averages to ensure the badness of the pixel
pix2 = np.zeros((1019,819))

for i in range(1019): #1024
    for j in range(819): #824
        if pix[i,j] == 1 and j > 5:
            sur = image[:,i-5:i+5,200+j-5:200+j+5]
            surmed = np.nanmedian(sur)
            surstd = np.nanstd(sur)
            pmed = np.nanmedian(image[:,i,200+j])
            pix2[i,j] = abs(surmed - pmed) > surstd
            
print(pix2)

## creating and saving fits

In [ ]:
#creating a workable data format for our boolean 2D Array
hdumy = fits.PrimaryHDU(pix2)

#Turning Workable Object into a fits
outd = fits.HDUList([hdumy])

## Construct a name and 'save directory' for the dark-subtracted, flatfielded image.
outname = 'bad_pixels' + '(INSERT NUMBER OR OTHER THING!!!!! OR ITLL BREAK SOME STUFF) 4' + '.fits'
outpath = savepath
print('outpath =',outpath)
outf = os.path.join(outpath,outname)
print('outname =',outname)

In [ ]:
## Now save the file (or not).
print(outf)
yes_or_no = input('Save file? Enter "y" or "n":')
if yes_or_no == 'y':
    outd.writeto(outf, overwrite = False)
    print( outname + ' has been saved.')
else:
    print( 'OK-- file was not saved.')

# outd.save(outf)

# Task 3 Masking bad pixels




In [ ]:
mask = np.zeros((len(files),1024,829))

for i in range (5,1019):
    for j in range (5,819):
        if pix2[i,j] == 1:
            sur = image[:,i-5:i+5,200+j-5:200+j+5]
            surmed = np.nanmedian(sur)
            mask[:,i,j] = image[:,i,j] - surmed
            

np.sum(mask) 

In [ ]:
maskhelp = np.zeros((len(files),1024,195))

In [ ]:
mask2 = np.append(maskhelp,mask, axis=2)

In [ ]:
smoothed = image-mask2

DO NOT RUN


In [ ]:
# # Display all the images in the stack.
# figx, figy = 20,20   # Use these to reveal fine detail in images.
figx, figy = 8,8   # Use these to get a quick look and save space.

for i in range(len(files)):
    plt.figure(figsize = (figx,figy))
    vmx= imedian[i] + madstd[i] * 3.0
    vmn = imedian[i] - madstd[i] * 3.0
#     grid()
    plt.title(files[i] + '   Median =' + str(imedian[i]) + '    mad_std =' + str(madstd[i]))
    plt.imshow(smoothed[i],'gray',interpolation = 'nearest',vmax=vmx,vmin=vmn)